In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")
id1 = df_train["id"]
id2 = df_test["id"]
df_train.drop(columns=["id"], inplace=True)
df_test.drop(columns=["id"], inplace=True)

In [3]:
from sklearn.model_selection import StratifiedKFold
sfk = StratifiedKFold()
def folds(df, set_):
    df["kfold"] = -1
    if set_ == "train":
        df["id"] = id1
        for f , (tid, vid) in enumerate(sfk.split(df, df["target"])):
            df.loc[vid, "kfold"] = f
    if set_ == "test":
        df["id"] = id2
        df
    return df

In [4]:
train_folds = folds(df_train, "train")
test = folds(df_test, "test")

In [5]:
test

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,kfold,id
0,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,0,...,-2.435736,-2.427430,-1.966887,5.734205,BAAABADLAC,99.478419,0,0,-1,900000
1,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,4,...,1.883322,-2.848714,-0.725155,3.194219,AFABBAEGCB,-65.993825,1,0,-1,900001
2,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,4,...,-5.157192,1.714005,0.585032,0.066898,BBACABBKEE,-87.405622,0,1,-1,900002
3,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,4,...,1.754570,-2.364007,-1.003320,3.893099,AEBEAACQCC,-281.293460,0,0,-1,900003
4,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,2,...,-0.130467,-3.557893,1.210687,1.861884,AEBBBBDABF,25.629415,0,2,-1,900004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,0.640110,0.897808,-0.523956,1.563760,-0.092281,-0.610867,0.535426,0,1,6,...,0.518110,1.243837,0.575111,0.076372,BCBCEBHMCD,204.186539,0,0,-1,1599995
699996,-0.191771,-0.035246,-0.118533,0.584750,2.126977,0.568659,-0.052663,4,3,4,...,-1.135740,2.982713,-1.511760,2.225218,BAABCADQFC,-97.694591,0,2,-1,1599996
699997,-0.331704,-0.328845,-1.185503,1.022128,-0.483099,-0.107146,-0.968281,1,1,2,...,1.096011,-2.734508,-4.885955,-2.248739,AAAJCBGQBA,130.622745,1,0,-1,1599997
699998,-2.031073,-1.238398,0.964699,-1.045950,0.906064,0.634301,-0.707474,5,1,1,...,1.018995,1.973697,-0.353068,-3.333449,BCBBCABNDE,-364.625148,0,0,-1,1599998


In [6]:
def reindex_df(df, excluding_columns):
    id_ = df.iloc[:, -excluding_columns:]
    cat = df.loc[:, df.dtypes=="int64"].iloc[:, :-excluding_columns]
    cont = df.loc[:, df.dtypes=="float64"]
    f27 = df.loc[:, df.dtypes=="object"]
    return pd.concat([id_,cat, cont, f27], axis=1)

In [7]:
train_folds = reindex_df(train_folds, 3)
test = reindex_df(test,2)

In [8]:
# alphabet dictionary
import string
i = 0
range_ = list(range(1,27))
alphabets = dict.fromkeys(string.ascii_uppercase,)
for name, key in alphabets.items():
    alphabets[name] = range_[i]
    i = i + 1

def f27_features(f27):
    features_split = []
    for id_ in f27.index:
        string = list(f27.loc[id_,"f_27"])
        features_split.append(string)

    encoded_features = []
    for feature in features_split:
        encoded = []
        for char in feature:
            encoded.append(alphabets[char])
        encoded_features.append(encoded)

    col_names = []
    for i in range(10):
        name = "f27_" + str(i)
        col_names.append(name)
    new_features = pd.DataFrame(encoded_features, columns=col_names)
    return new_features

In [9]:
new_features = f27_features(train_folds)
train_folds = pd.concat([train_folds, new_features], axis=1)
train_folds.drop(columns=["f_27"], inplace=True)

In [10]:
import lightgbm
from sklearn.metrics import accuracy_score
def objective(trial, df):
    params = {
    'objective':'binary',
    'metric' : 'binary_error',
    'boosting': 'gbdt',
    'num_leaves': trial.suggest_int("num_leaves",2 , 256),
    'feature_fraction': trial.suggest_float("feature_fraction", 0.1, 1.0),
    'bagging_fraction': trial.suggest_float("bagging_fraction", 0.1, 1.0),
    'learning_rate': trial.suggest_float("learning_rate", 0.01,1) }
    accuracy = []
    for fold in range(5):
        train = df[df["kfold"] != fold]
        valid = df[df["kfold"] == fold]
        train_lgbm = lightgbm.Dataset(train.iloc[:,3:].to_numpy(), train["target"].to_numpy())
        model_lgbm = lightgbm.train(params, train_lgbm, num_boost_round=200)
        pred_prob = model_lgbm.predict(valid.iloc[:,3:].to_numpy())
        y_pred = []
        for pred in pred_prob:
            if pred >= 0.5:
                y_pred.append(1)
            else:
                y_pred.append(0)
        ypred = np.array(y_pred)
        fold_accuracy = accuracy_score(valid["target"].to_numpy(), ypred)
        accuracy.append(fold_accuracy)
        return np.mean(accuracy)

In [11]:
from functools import partial
optimisation_func = partial(objective, df=train_folds)

In [12]:
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(optimisation_func, n_trials=100)

[I 2022-05-28 07:37:11,645] A new study created in memory with name: no-name-759252e7-500b-42af-82f7-d9d220e048f3


[LightGBM] [Info] Number of positive: 350272, number of negative: 369728
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4364
[LightGBM] [Info] Number of data points in the train set: 720000, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.486489 -> initscore=-0.054058
[LightGBM] [Info] Start training from score -0.054058


[I 2022-05-28 07:37:28,189] Trial 0 finished with value: 0.8897722222222222 and parameters: {'num_leaves': 177, 'feature_fraction': 0.31999749666241584, 'bagging_fraction': 0.6199122588388134, 'learning_rate': 0.1680817809171493}. Best is trial 0 with value: 0.8897722222222222.


[LightGBM] [Info] Number of positive: 350272, number of negative: 369728
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4364
[LightGBM] [Info] Number of data points in the train set: 720000, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.486489 -> initscore=-0.054058
[LightGBM] [Info] Start training from score -0.054058


[I 2022-05-28 07:37:39,046] Trial 1 finished with value: 0.8460944444444445 and parameters: {'num_leaves': 4, 'feature_fraction': 0.5820646456046967, 'bagging_fraction': 0.6564109377014177, 'learning_rate': 0.8705374895181737}. Best is trial 0 with value: 0.8897722222222222.


...

[I 2022-05-28 08:14:03,600] Trial 97 finished with value: 0.9072111111111111 and parameters: {'num_leaves': 251, 'feature_fraction': 0.9795159079141189, 'bagging_fraction': 0.25545572009164447, 'learning_rate': 0.665459393765498}. Best is trial 38 with value: 0.9217666666666666.


[LightGBM] [Info] Number of positive: 350272, number of negative: 369728
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055286 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4364
[LightGBM] [Info] Number of data points in the train set: 720000, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.486489 -> initscore=-0.054058
[LightGBM] [Info] Start training from score -0.054058


[I 2022-05-28 08:14:26,610] Trial 98 finished with value: 0.91605 and parameters: {'num_leaves': 170, 'feature_fraction': 0.9998071091068964, 'bagging_fraction': 0.4310056826984038, 'learning_rate': 0.14773242627418123}. Best is trial 38 with value: 0.9217666666666666.


[LightGBM] [Info] Number of positive: 350272, number of negative: 369728
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050689 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4364
[LightGBM] [Info] Number of data points in the train set: 720000, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.486489 -> initscore=-0.054058
[LightGBM] [Info] Start training from score -0.054058


[I 2022-05-28 08:14:47,345] Trial 99 finished with value: 0.8993 and parameters: {'num_leaves': 232, 'feature_fraction': 0.9225277082443837, 'bagging_fraction': 0.598286205857153, 'learning_rate': 0.8263159370802345}. Best is trial 38 with value: 0.9217666666666666.


In [13]:
best_params = study.best_params

In [14]:
best_params

{'num_leaves': 255,
 'feature_fraction': 0.9646367375935679,
 'bagging_fraction': 0.19053681610791118,
 'learning_rate': 0.16513221476967985}

In [15]:
new_features = f27_features(test)
test = pd.concat([test, new_features], axis=1)
test.drop(columns=["f_27"], inplace=True)

In [16]:
test.columns

Index(['kfold', 'id', 'f_07', 'f_08', 'f_09', 'f_10', 'f_11', 'f_12', 'f_13',
       'f_14', 'f_15', 'f_16', 'f_17', 'f_18', 'f_29', 'f_30', 'f_00', 'f_01',
       'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19', 'f_20', 'f_21', 'f_22',
       'f_23', 'f_24', 'f_25', 'f_26', 'f_28', 'f27_0', 'f27_1', 'f27_2',
       'f27_3', 'f27_4', 'f27_5', 'f27_6', 'f27_7', 'f27_8', 'f27_9'],
      dtype='object')

In [17]:
x = train_folds.iloc[:, 3:].to_numpy()
y = train_folds["target"].to_numpy()
final_train_set = lightgbm.Dataset(x,y)
params = {
    'objective':'binary',
    'metric': 'binary_error',
    'boosting':'gbdt',
    'n_estimators':1000,
    'num_leaves':best_params["num_leaves"],
    'feature_fraction':best_params["feature_fraction"],
    'bagging_fraction':best_params["bagging_fraction"],
    'learning_rate':best_params["learning_rate"],
    'verbose':1
}
fitted_final_model = lightgbm.train(params, final_train_set)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 437839, number of negative: 462161
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.064789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4366
[LightGBM] [Info] Number of data points in the train set: 900000, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.486488 -> initscore=-0.054062
[LightGBM] [Info] Start training from score -0.054062


In [18]:
test_ = test.iloc[:, 2:].to_numpy()
ypred = fitted_final_model.predict(test_)
'''y_pred = []
for pred in ypred:
    if pred >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
'''

'y_pred = []\nfor pred in ypred:\n    if pred >= 0.5:\n        y_pred.append(1)\n    else:\n        y_pred.append(0)\n'

In [19]:
submission = pd.DataFrame({ 'id': test["id"],
                            'target': ypred})

In [20]:
submission.to_csv("submission.csv", index=False)